In [1]:
def grayCode(n): 
  
    # Right Shift the number 
    # by 1 taking xor with  
    # original number 
    grayval = n ^ (n >> 1) 
    
    return int(bin(grayval)[2:] )

def graydecode(binary):
    
    # binary -> decimal
    

    binary1 = binary
    decimal, i, n = 0, 0, 0
    while(binary != 0): 
        dec = binary % 10
        decimal = decimal + dec * pow(2, i) 
        binary = binary//10
        i += 1
    
    # Taking xor until 
    # n becomes zero 
    inv = 0
    while(decimal): 
        inv = inv ^ decimal; 
        decimal = decimal >> 1;

    return inv 

def num2gray(n,gray_len):
    gy = str(grayCode(n))
    
    if len(gy) < gray_len:
        gy = '0'* (gray_len-len(gy)) + gy
        
    return gy

In [2]:
'0'*3

'000'

In [3]:
grayCode(99)

1010010

In [4]:
graydecode(1000000)

127

### Gray Code Size 정하기
* 100개 몇 자리 수가 필요한지 알아야 고정된 크기의 individual 생산 가능

1. num_graph -> 필요한 자리수 추출
2. 필요한 자리수를 설정
3. 자리수 보다 적으면 앞에 0 붙여 주기
4. string 형으로 해야 관리가 된다.

In [11]:
num_graph = 100
gray_len = len(str(grayCode(128-1)))

In [12]:
gray_len

7

In [13]:
num2gray(2,gray_len)

'0000011'

### Gray Code Crossover
* individual를 gray code로 구성
* 각 구획 별로 Crossover 실시 // 일정 확률 이상 (0.7) 일 때만 섞기 독립적으로

In [14]:
from __future__ import division
import random
import warnings

try:
    from collections.abc import Sequence
except ImportError:
    from collections import Sequence

from itertools import repeat

def cxOnePoint(ind1, ind2):
    """Executes a one point crossover on the input :term:`sequence` individuals.
    The two individuals are modified in place. The resulting individuals will
    respectively have the length of the other.
    :param ind1: The first individual participating in the crossover.
    :param ind2: The second individual participating in the crossover.
    :returns: A tuple of two individuals.
    This function uses the :func:`~random.randint` function from the
    python base :mod:`random` module.
    """
    size = min(len(ind1), len(ind2))
    cxpoint = random.randint(1, size - 1)
    ind1[cxpoint:], ind2[cxpoint:] = ind2[cxpoint:], ind1[cxpoint:]

    return ind1, ind2


def cxgray(ind1,ind2,num_graph):
    
    gray_len = len(str(grayCode(num_graph)))-1
    
    ## gray_len : 한 개 gray code의 길이, 전체 //3 
    new_ind1 = []
    new_ind2 = []
    
    for i in range(3):
        temp_ind1 = ind1[gray_len*i:gray_len*(i+1)]
        temp_ind2 = ind2[gray_len*i:gray_len*(i+1)]
        print(temp_ind1)
        if random.random() < 0.7 :
            x1,x2 = cxOnePoint(temp_ind1,temp_ind2)
            new_ind1.extend(x1)
            new_ind2.extend(x2)
        else:
            new_ind1.extend(temp_ind1)
            new_ind2.extend(temp_ind2)
    
    return new_ind1, new_ind2

### Gray Code 와 기존 차이점
* Cross over 함수가 다름!
* individual를 읽고 쓸 때 encode decode를 해줘야 / individual의 크기
* hyperparameter에 graycode 유무로 넣어주기
* 만약 graycode값이 100을 초과하면 값 랜덤으로 바꾸기? -. 100으로 넣기? // evaluation 전에 항상 확인하기! --> 여기 문제에 봉착, 기존에 있는 요소를 어떻게 대체할 것 인가 --> del 이용해주자


In [15]:
random.random()

0.7215265121948701

In [18]:
import os
import sys
import os
import logging
from easydict import EasyDict
import numpy as np
import random
import time
import datetime
from deap import tools
from collections import OrderedDict
from pprint import pprint
import json
import torch
from copy import deepcopy

sys.path.insert(0, '../')
from utils_kyy.utils_graph import make_random_graph
from utils_kyy.create_toolbox import create_toolbox_for_NSGA_RWNN
from utils_kyy.create_toolbox import evaluate
from utils_kyy.utils_graycode import *


class rwns_train:
    def __init__(self, json_file):
        self.root = os.path.abspath(os.path.join(os.getcwd(), '..'))
        #self.root = os.getcwd()
        self.param_dir = os.path.join(self.root + '/parameters/', json_file)
        f = open(self.param_dir)
        params = json.load(f)
        pprint(params)
        self.name = params['NAME']
        self.log_dir = os.path.join(self.root, 'log')
        self.model_dir = os.path.join(self.root +'/saved_models/', self.name)
        
        
        # create directory
        if not (os.path.isdir(self.model_dir)):
            os.makedirs(self.model_dir)
        if not (os.path.isdir(self.log_dir)):
            os.makedirs(self.log_dir)


        ## toolbox params
        self.args_train = EasyDict(params['ARGS_TRAIN'])
        self.data_path = params['DATA_PATH']
        self.run_code = params['RUN_CODE']
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '/'
        self.num_graph = params['NUM_GRAPH']
        self.toolbox = None

        ## GA params
        self.pop_size = params['POP_SIZE']
        self.ngen = params['NGEN']
        self.cxpb = params['CXPB']
        self.mutpb = params['MUTPB']

        ## logs
        log = OrderedDict()

        log['hp'] = self.args_train
        self.log = log
        self.train_log = None 
        
        
        
    ## tool box and make graph
    def create_toolbox(self):
        self.stage_pool_path = '../graph_pool' + '/' + self.run_code + '/'
        self.log_path = '../logs/' + self.run_code + '_' + self.name + '/'

        if not os.path.exists(self.stage_pool_path): os.makedirs(self.stage_pool_path)
        if not os.path.isdir(self.log_path): os.makedirs(self.log_path)
        self.log_file_name = self.log_path + 'logging.log'
        self.train_log_file_name = self.log_path + 'train_logging.log'

        logging.basicConfig(filename=self.log_file_name, level=logging.INFO)
        logging.info('Start to write log.')

        # num_graph = 100
        make_random_graph(self.num_graph, self.stage_pool_path)

        return create_toolbox_for_NSGA_RWNN(self.num_graph, self.args_train, self.stage_pool_path, self.data_path,
                                            self.log_file_name)

    ## Train

    def train(self):

                
        ## Parameters
        self.create_toolbox()
        POP_SIZE = self.pop_size

        ## train log
        train_log = OrderedDict()

        if self.toolbox is None:
            self.toolbox = self.create_toolbox()
        
        if self.args_train.graycode:
            gray_len = len(str(grayCode(self.num_graph)))

        toolbox = self.toolbox

        # log에 기록할 stats
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("min", np.min, axis=0)
        stats.register("max", np.max, axis=0)

        logbook = tools.Logbook()
        logbook.header = "gen", "evals", "min", "max", "evals_time", "gen_time"

        # population 생성.  (toolbox.population은 creator.Individual n개를 담은 list를 반환. (=> population)
        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialion starts ...")
        logging.info("Initialion starts at " + now_str)
        init_start_time = time.time()

        pop = toolbox.population(n=POP_SIZE)

        ## fitness, model list
        fit_list = []
        model_list = []
        
        local_min_fit1 = float('inf')
        local_min_fit2 = float('inf')
        local_min_index = [None, None]
        
        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in pop if not ind.fitness.valid]
        #fitnesses = toolbox.map(toolbox.evaluate, invalid_ind)  # .evaluate는 tuple을 반환. 따라서 fitnesses는 튜플을 원소로 가지는 list
    
#         checked_ind = []
        for idx, ind in enumerate(invalid_ind):
        
        
#             if self.args_train.graycode: 
#                 for k in range(3):
#                     checked_ind.extend(check_Upper(ind[gray_len*k:gray_len*(k+1)],self.num_graph))
                
#                 print("Check_ind",checked_ind)
#                 print("Check_ind",ind)
#                 ind = deepcopy(checked_ind)
                
            #print("ind",ind)
            fitness, ind_model = evaluate(ind,args_train=self.args_train, stage_pool_path=self.stage_pool_path,data_path=self.data_path ,log_file_name=self.log_file_name)
            ind.fitness.values = fitness
            fit_list.append(fitness)
            model_list.append(ind_model)
            
            if fitness[0] < local_min_fit1:
                local_min_fit1 = fitness[0]
                local_min_index[0] = idx

            if fitness[1] < local_min_fit2:
                local_min_fit2 = fitness[1]
                local_min_index[1] = idx

        ## index ckpt download
        print("#### Saving Model", local_min_index)
        self.save_model(model=model_list[local_min_index[0]],ngen=0,subname=str(0)+'_'+'acc')                          
        self.save_model(model=model_list[local_min_index[1]],ngen=0,subname=str(0)+'_'+'flops')                          

            
        ## log 기록
        train_log[0] = fit_list
        self.train_log = train_log

        # This is just to assign the crowding distance to the individuals
        # no actual selection is done
        pop = toolbox.select(pop, len(pop))

        record = stats.compile(pop)
        logbook.record(gen=0, evals=len(invalid_ind), **record)
        print(logbook.stream)

        now = datetime.datetime.now()
        now_str = now.strftime('%Y-%m-%d %H:%M:%S')
        print("Initialization is finished at", now_str)
        logging.info("Initialion is finished at " + now_str)

        init_time = time.time() - init_start_time
        logging.info("Initialion time = " + str(init_time) + "s")

        print()

        # Begin the generational process
        for gen in range(1, self.ngen):
            now = datetime.datetime.now()
            now_str = now.strftime('%Y-%m-%d %H:%M:%S')
            print("#####", gen, "th generation starts at", now_str)
            logging.info(str(gen) + "th generation starts at" + now_str)

            start_gen = time.time()
            # Vary the population
            offspring = tools.selTournamentDCD(pop, len(pop))
            offspring = [toolbox.clone(ind) for ind in offspring]

            for ind1, ind2 in zip(offspring[::2], offspring[1::2]):
                if random.random() <= self.cxpb:
                    toolbox.mate(ind1, ind2)

                toolbox.mutate(ind1, indpb=self.mutpb)
                toolbox.mutate(ind2, indpb=self.mutpb)
                del ind1.fitness.values, ind2.fitness.values

            # Evaluate the individuals with an invalid fitness
            print("##### Evaluation starts")
            start_time = time.time()

            ## fitness value : accuracy ,flops 모음
            fit_list = []
            model_listist = []

            ## 가장 최소 1,2 구하기
            local_min_fit1 = float('inf')
            local_min_fit2 = float('inf')
            local_min_index = [None, None]

            invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
                 
            check_ind = []
            
            for idx,ind in enumerate(invalid_ind):
            
            
#                 if self.args_train.graycode:
#                     for k in range(3):
#                         checked_ind.extend(check_Upper(ind[gray_len*k:gray_len*(k+1)],self.num_graph))

#                     ind = deepcopy(checked_ind)


                fitness, ind_model = evaluate(ind,args_train=self.args_train, stage_pool_path=self.stage_pool_path,data_path=self.data_path ,log_file_name=self.log_file_name)
                ind.fitness.values = fitness
                fit_list.append(fitness)
                model_list.append(ind_model)

                if fitness[0] < local_min_fit1:
                    local_min_fit1 = fitness[0]
                    local_min_index[0] = idx

                if fitness[1] < local_min_fit2:
                    local_min_fit2 = fitness[1]
                    local_min_index[1] = idx


            ## index ckpt download
            print("#### Saving Model", local_min_index)
            self.save_model(model=model_list[local_min_index[0]],ngen=gen,subname=str(gen)+'_'+'acc')                          
            self.save_model(model=model_list[local_min_index[1]],ngen=gen,subname=str(gen)+'_'+'flops')                          

            ## log 기록
            train_log[gen] = fit_list
            self.train_log = train_log

            eval_time_for_one_generation = time.time() - start_time
            print("##### Evaluation ends (Time : %.3f)" % eval_time_for_one_generation)

            # Select the next generation population
            pop = toolbox.select(pop + offspring, POP_SIZE)

            gen_time = time.time() - start_gen
            print('##### [gen_time: %.3fs]' % gen_time, gen, 'th generation is finished.')

            record = stats.compile(pop)
            logbook.record(gen=gen, evals=len(invalid_ind), **record,
                           evals_time=eval_time_for_one_generation, gen_time=gen_time)

            logging.info('Gen [%03d/%03d] -- evals: %03d, evals_time: %.4fs, gen_time: %.4fs' % (
            gen, self.ngen, len(invalid_ind), eval_time_for_one_generation, gen_time))
            print(logbook.stream)

    ## Save Check point

    ## Save Log
    def save_log(self):

        log = self.log

        ## 필요한 log 추후 정리하여 추l가
        log['train_log'] = self.train_log

        with open(self.train_log_file_name, 'w', encoding='utf-8') as make_file:
            json.dump(log, make_file, ensure_ascii=False, indent='\t')

    ## Save Model
    def save_model(self, model,ngen, subname):
        
        model_fname = self.name + '_' + str(ngen) + '_' + subname
        model_path = os.path.join(self.model_dir, model_fname)
        print("Saving Model",model_path)
        torch.save(model,model_path)


In [19]:
trainer = rwns_train("gray_code_default.json")
trainer.create_toolbox()
trainer.train()
trainer.save_log()

{'ARGS_TRAIN': {'base_lr': 0.1,
                'batch_size': 128,
                'data': 'CIFAR10',
                'epochs': 5,
                'graycode': True,
                'input_dim': 3,
                'lr_mode': 'cosine',
                'momentum': 0.9,
                'num_classes': 10,
                'print_freq': 100,
                'targetlr': 0.0,
                'warmup_epochs': 0,
                'warmup_lr': 0.0,
                'warmup_mode': 'linear',
                'weight_decay': 5e-05,
                'workers': 4},
 'CXPB': 0.5,
 'DATA_PATH': 'D:/data/cifar10/',
 'MUTPB': 0.5,
 'NAME': 'gray_Code_default',
 'NGEN': 50,
 'NUM_GRAPH': 128,
 'POP_SIZE': 4,
 'RUN_CODE': 'test_kyy_CIFAR10_time_check'}
Start to make random graph pool...
Finished


C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\Users\82103\Anaconda3\envs\newrw\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Start to make random graph pool...
Finished
Start to make random graph pool...
Finished
Initialion starts ...
1 71 19
Files already downloaded and verified
Files already downloaded and verified


RuntimeError: CUDA out of memory. Tried to allocate 20.88 MiB (GPU 0; 8.00 GiB total capacity; 5.93 GiB already allocated; 5.19 MiB free; 25.30 MiB cached)

In [16]:
a = [0,2,3,4]

In [17]:
a.del(0:2)

SyntaxError: invalid syntax (<ipython-input-17-20d45705f8cb>, line 1)

In [20]:
del a[:2]

In [21]:
a

[3, 4]

In [20]:
import argparse